# Processing mixed beverage data
This Jupyter Notebook uses [data.texas.gov Mixed Beverage Gross Receipts data](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This is a work in progress.

- The first version will use a downloaded file to process. The data is in a new format, so it has to be reworked
- The second phase will work on pulling the data directly from Socrata.

### This links may or may not work
- [Top sales statewide](#Top-sales-statewide)
- [Austin sales](#Austin-sales-and-sums)
- [Central Texas cities](#More-Central-Texas-cities)



I'll use a little bash here to look at the file I have pulled, which is files with `Obligation End Date` between Sept. 1 and Sept 30 2017.


In [1]:
%%bash
head -n 5 data-raw/Mixed_Beverage_Gross_Receipts-201709.csv

Taxpayer Number,Taxpayer Name,Taxpayer Address,Taxpayer City,Taxpayer State,Taxpayer Zip,Taxpayer County,Location Number,Location Name,Location Address,Location City,Location State,Location Zip,Location County,Inside/Outside City Limits,TABC Permit Number,Responsibility Begin Date,Responsibility End Date,Obligation End Date,Liquor Receipts,Wine Receipts,Beer Receipts,Cover Charge Receipts,Total Receipts
32052221093,"CTWL, LLC",2712 BEE CAVES RD STE 124,AUSTIN,TX,78746,227,1,CHINATOWN,2712 BEE CAVES RD STE 124,ROLLINGWOOD,TX,78746,227,Y,MB866922,03/20/2014,,09/30/2017,$8727,$12270,$2940,$0,$23937
32050880221,"ISB ENTERPRISE, LLC.",3659 NASA PKWY,SEABROOK,TX,77586,101,2,SIGNATURES,3659 NASA PKWY,SEABROOK,TX,77586,101,Y,MB851753,09/01/2016,,09/30/2017,$3710,$5067,$3582,$0,$12359
32040734306,"MAMA IRMA'S, INC.",7325 SPENCER HWY,PASADENA,TX,77505,101,1,MAMA IRMA'S,7325 SPENCER HWY,PASADENA,TX,77505,101,Y,MB764780,01/04/2011,,09/30/2017,$46186,$420,$18875,$0,$65481
32036703190,CAFE HOLDINGS 

This data appears to be MUCH cleaner than what we have worked with in the past. It looks like we won't have to trim fields.

Now that we have our file and know what it looks like, we'll use Python and the agate library to clean and analyze it. You'll need to make sure that you have agate installed, preferably in an virtual environment like Conda, as described in the [ReadMe](README.md).

In [2]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

/Users/christian/anaconda/envs/mixbev/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-16 22:00:51.007718. Please add timezone info to timestamps.


In [3]:
# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Study variables

This is where you set which file you are working with, and which month you want to study, etc.

First, we'll list the files in our directory that we have downloaded so far so we can get the filename:

In [4]:
ls data-raw/

Mixed_Beverage_Gross_Receipts-201709.csv


Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

In [5]:
# this is our source file, which may have been downloaded above
file = 'data-raw/Mixed_Beverage_Gross_Receipts-201709.csv'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

# setting the month_studied var.
# This should be checked in the table below that counts records by month
#THIS MIGHT NEED TO CHANGE ONCE I UNDERSTAND THE NEW DATA
month_studied = '2017/03'


### Import the file
There are a couple of things we have to set to import a file. Remember we had to do the same when we did this manually in Tableau.
- Set the column header names (BUT NEW DATA HAS HEADERS)
- Set the ZIP and County codes as text, so we preserve '001'.
- The encoding type of the file (Tableau enferred, but we have to specify here. Common types are 'iso-8859-1' or  'utf-8', or 'latin1'. Just try it until it works.

In [6]:
#Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Taxpayer Number': agate.Text(),
    'Location Number': agate.Text(),
    'Taxpayer Zip': agate.Text(),
    'Location Zip': agate.Text(),
    'Location County': agate.Text(),
    'Taxpayer County': agate.Text()
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, encoding='iso-8859-1', column_types=specified_types)

# prints table fields so we an check thoes data types
print(mixbev_raw)

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [7]:
# printing table to see results
mixbev_raw.print_table()


| Taxpayer Number | Taxpayer Name        | Taxpayer Address     | Taxpayer City | Taxpayer State | Taxpayer Zip | ... |
| --------------- | -------------------- | -------------------- | ------------- | -------------- | ------------ | --- |
| 32052221093     | CTWL, LLC            | 2712 BEE CAVES RD... | AUSTIN        | TX             | 78746        | ... |
| 32050880221     | ISB ENTERPRISE, LLC. | 3659 NASA PKWY       | SEABROOK      | TX             | 77586        | ... |
| 32040734306     | MAMA IRMA'S, INC.    | 7325 SPENCER HWY     | PASADENA      | TX             | 77505        | ... |
| 32036703190     | CAFE HOLDINGS PRI... | 9660 AUDELIA RD S... | DALLAS        | TX             | 75238        | ... |
| 32049968608     | SHYSG GROUP, LLC     | 2686 BLACK BEAR DR   | NEW BRAUNFELS | TX             | 78132        | ... |
| 12039294405     | STUDIO CLUB 4, LLC   | 12404 PARK CENTRA... | DALLAS        | TX             | 75251        | ... |
| 17521347561     | T AND N, INCORPOR...

## no cleaning?
- Unlike befor, we don't need to trim fields or calculate the Gross Receipts from Reported Tax, becaues the are already gross
- If we are interested in Tax paid, we will have to create a function to get those based on the tax rate, which will vary by the date of the report (there are two, which are outlined on the record layout linked from the [data primer](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893)

### Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [8]:
# Concatenates the name and address
mixbev_establishment = mixbev_raw.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(Location Name)s (%(Location Address)s)' % row))
])

# Prints columns so you see it is there
print(mixbev_establishment)

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [9]:
# selects and prints Establishment to check what is looks like
mixbev_establishment.select('Establishment').limit(5).print_table(max_column_width=80)

| Establishment                                             |
| --------------------------------------------------------- |
| CHINATOWN (2712 BEE CAVES RD STE 124)                     |
| SIGNATURES (3659 NASA PKWY)                               |
| MAMA IRMA'S (7325 SPENCER HWY)                            |
| OFFSHORE'S NEXTDOOR (9660 AUDELIA RD STE 305)             |
| LOUIE'S BEER GARDEN AND OYSTER HOUSE (119 E HUTCHISON ST) |


### Import and merge counties lookup table
We do this to get county names. I got this list from the comptroller.

NOTE: Wisdom would suggest we join on the `code` column from counites, but the data.texas.gov data does not have the zero padding from those values, so I'm using the `id` column.

In [10]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('counties.csv', column_types={
    'code': agate.Text(),
    'id': agate.Text()
})

# peek at the column names
print(counties)

| column | data_type |
| ------ | --------- |
| id     | Text      |
| county | Text      |
| code   | Text      |



In [11]:
# peek at the data
counties.limit(5).print_table()
print(counties)

| id | county   | code |
| -- | -------- | ---- |
| 1  | Anderson | 001  |
| 2  | Andrews  | 002  |
| 3  | Angelina | 003  |
| 4  | Aransas  | 004  |
| 5  | Archer   | 005  |
| column | data_type |
| ------ | --------- |
| id     | Text      |
| county | Text      |
| code   | Text      |



In [12]:
# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_establishment.join(counties, 'Location County', 'id')

# check that the merge was succesful 
print(mixbev_joined)
mixbev_joined.print_table()

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [13]:
# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Location Name',
    'Location Address',
    'Establishment',
    'Location City',
    'Location State',
    'Location Zip',
    'county',
    'Total Receipts',
    'Obligation End Date'
]).rename(column_names = {
    'Location Name' : 'Name',
    'Location Address' : 'Address',
    'Location City': 'City',
    'Location State': 'State',
    'Location Zip': 'Zip',
    'Total Receipts' : 'Receipts',
    'county': 'County',
    'Obligation End Date': 'Report date'
})

# peek at the column names
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| Zip           | Text      |
| County        | Text      |
| Receipts      | Number    |
| Report date   | Date      |



In [14]:
# peek at the table
mixbev.limit(5).print_table()

| Name                 | Address              | Establishment        | City        | State | Zip   | ... |
| -------------------- | -------------------- | -------------------- | ----------- | ----- | ----- | --- |
| CHINATOWN            | 2712 BEE CAVES RD... | CHINATOWN (2712 B... | ROLLINGWOOD | TX    | 78746 | ... |
| SIGNATURES           | 3659 NASA PKWY       | SIGNATURES (3659 ... | SEABROOK    | TX    | 77586 | ... |
| MAMA IRMA'S          | 7325 SPENCER HWY     | MAMA IRMA'S (7325... | PASADENA    | TX    | 77505 | ... |
| OFFSHORE'S NEXTDOOR  | 9660 AUDELIA RD S... | OFFSHORE'S NEXTDO... | DALLAS      | TX    | 75238 | ... |
| LOUIE'S BEER GARD... | 119 E HUTCHISON ST   | LOUIE'S BEER GARD... | SAN MARCOS  | TX    | 78666 | ... |


## No need to view dates

Previously we had to filter down to only the dates studied. As this stands now, we filtered to the correct date in Socrata BEFORE we exported. This might need to change once we pull from the API.

In [15]:
# double-checking I'm looking at one report data
mixbev_dates = mixbev.select('Report date').distinct('Report date')
mixbev_dates.print_table()

print('\nNumber of records in table: {}'.format(
        len(mixbev))
     )

| Report date |
| ----------- |
|  2017-09-30 |

Number of records in table: 15536


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to computer the Tax and Receipts columns.

In [16]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=40)

| Establishment                            | County  | City        | Sales_sum |
| ---------------------------------------- | ------- | ----------- | --------- |
| AT&T STADIUM (1 LEGENDS WAY)             | Tarrant | ARLINGTON   | 3,883,207 |
| GAYLORD TEXAN (1501 GAYLORD TRL)         | Tarrant | GRAPEVINE   | 1,739,696 |
| HOSPITALITY INTERNATIONAL, INC. (2380... | Bexar   | SAN ANTONIO | 1,376,321 |
| WLS BEVERAGE CO (110 E 2ND ST)           | Travis  | AUSTIN      | 1,100,292 |
| OMNI DALLAS CONVENTION CENTER (555 S ... | Dallas  | DALLAS      |   983,038 |
| ARAMARK SPORTS & ENTERTAINMENT SERVIC... | Harris  | HOUSTON     |   948,918 |
| METROPLEX SPORTSERVICE, INC. (1000 BA... | Tarrant | ARLINGTON   |   902,437 |
| RYAN SANDERS SPORTS SERVICES, LLC (92... | Travis  | DEL VALLE   |   873,760 |
| HAPPIEST HOUR, LLC (2616 OLIVE ST)       | Dallas  | DALLAS      |   830,658 |
| SALC, INC. (2201 N STEMMONS FWY FL 1)    | Dallas  | DALLAS      |   801,019 |


## Overall statewide sum

In [17]:
# summing sales statewide for month

print('Totalstatewide sales for this month are: {}'.format(
    mixbev.aggregate(agate.Sum('Receipts'))
))



Totalstatewide sales for this month are: 572120305


## Location sums function

Because we want to get the top sellers in a bunch of cities and couties, we create a function so we don't have to repeat the code. This function allows us to pass in a city or county name to filter the monthly receipts table and then sum the Tax and Receipts columns. The result can then be acted on to print or aggreggate.

In [18]:
# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [19]:
# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')

# print the resulting table
austin.limit(5).print_table(max_column_width=60)

| Establishment                                                | City   | Receipts_sum |
| ------------------------------------------------------------ | ------ | ------------ |
| WLS BEVERAGE CO (110 E 2ND ST)                               | AUSTIN |    1,100,292 |
| ROSE ROOM/ 77 DEGREES (11500 ROCK ROSE AVE)                  | AUSTIN |      585,686 |
| 400 BAR/CUCARACHA/CHUPACABRA/JACKALOPE/MOOSENUCKLE (400 E... | AUSTIN |      503,352 |
| BLIND PIG PUB / PIG PEN (317 E 6TH ST)                       | AUSTIN |      502,440 |
| W HOTEL AUSTIN (200 LAVACA ST)                               | AUSTIN |      441,207 |


### String methods together to print a table
So far, we've been printing tables, but Agate can also print charts. The **`print_bars`** method creates a simple, text-based bar chart.

In [20]:
# We'll use the same function, but instead of creating a new table,
# we'll just string on the limit and print bars methods
# print_bars needs to arguments, the label column and then the value to make the chart from
location_sum('City', 'AUSTIN').limit(10).print_bars('Establishment', 'Receipts_sum', width=90)

Establishment                                                     Receipts_sum
WLS BEVERAGE CO (110 E 2ND ST)                                       1,100,292 ▓░░░░░░    
ROSE ROOM/ 77 DEGREES (11500 ROCK ROSE AVE)                            585,686 ▓░░░       
400 BAR/CUCARACHA/CHUPACABRA/JACKALOPE/MOOSENUCKLE (400 E 6TH ST)      503,352 ▓░░░       
BLIND PIG PUB / PIG PEN (317 E 6TH ST)                                 502,440 ▓░░░       
W HOTEL AUSTIN (200 LAVACA ST)                                         441,207 ▓░░        
THE DOGWOOD DOMAIN (11420 ROCK ROSE AVE STE 700)                       398,332 ▓░░        
HOTEL VAN ZANDT (605 DAVIS ST)                                         393,394 ▓░░        
THE PALAZIO (501 E BEN WHITE BLVD)                                     376,713 ▓░░        
CONTAINER BAR (90 RAINEY ST)                                           375,494 ▓░░        
TOP GOLF (2700 ESPERANZA XING)                                         367,731 ▓░░        
           

### Total sales Austin

In [21]:
# Austin total sales as s city
# This sums the grouped table, but it works

print('Total sales in Austin are: {}'.format(
    
))



IndexError: tuple index out of range

## More Central Texas cities

In [ ]:
location_sum('City', 'BASTROP').limit(5).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=60)

In [ ]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=60)

## Sales by county

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [ ]:
location_sum('County', 'BASTROP').limit(5).print_table(max_column_width=80)

In [ ]:
location_sum('County', 'CALDWELL').limit(3).print_table(max_column_width=80)

In [ ]:
location_sum('County', 'HAYS').limit(3).print_table(max_column_width=80)

In [ ]:
location_sum('County', 'TRAVIS').limit(3).print_table(max_column_width=80)

In [ ]:
location_sum('County', 'WILLIAMSON').limit(3).print_table(max_column_width=80)

## Sales by ZIP Code
For this, we'll do a simple pivot on Zip, but instead of using the default Count method, we'll pass in an aggregation to add the Tax values together for all the Zips. Whe then order_by in reverse to get the top values.

In [ ]:
# top zip code gross receipts
zip_receipts = mixbev.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()